In [5]:
import sqlite3
import os

from pathlib import Path

# Obtain the execution path
actual_path = Path.cwd()

# Obtain the parent directory
father_dir = actual_path.parent
la_liga_dataset = os.path.join(father_dir, "laliga.sqlite")
print(la_liga_dataset)

C:\Users\marre\GitHubRepos\la-quiniela\laliga.sqlite


In [15]:
# Connect to the database
conexion = sqlite3.connect(la_liga_dataset)

# Create cursor to be able to ask data
cursor = conexion.cursor()

# Select the Matches table
cursor.execute("SELECT * FROM Matches")  # cambia 'usuarios' por tu tabla

# Obtain all the data
filas = cursor.fetchall()

# Close the connection
conexion.close()

In [16]:
type(filas)

list

In [28]:
import pandas as pd
columns = ["season", "division", "matchday", "date", "time",
            "home_team", "away_team", "score"]
df = pd.DataFrame(filas, columns=columns)

In [29]:
df

,season,division,matchday,date,time,home_team,away_team,score
0,1928-1929,1,1,2/10/29,None,Arenas Club,Athletic Madrid,2:3
1,1928-1929,1,1,2/10/29,None,Espanyol,Real Unión,3:2
2,1928-1929,1,1,2/10/29,None,Real Madrid,Catalunya,5:0
3,1928-1929,1,1,2/10/29,None,Donostia,Athletic,1:1
4,1928-1929,1,1,2/12/29,None,Racing,Barcelona,0:2
...,...,...,...,...,...,...,...,...
48775,2021-2022,2,42,5/29/22,None,Real Oviedo,UD Ibiza,None
48776,2021-2022,2,42,5/29/22,None,Real Sociedad B,Real Zaragoza,None
48777,2021-2022,2,42,5/29/22,None,Sporting Gijón,UD Las Palmas,None
48778,2021-2022,2,42,5/29/22,None,CD Tenerife,FC Cartagena,None


In [32]:
df_first = df[df["division"] == 1]

In [39]:
df_first = df_first.dropna(subset=["score"])

In [40]:
df_first

,season,division,matchday,date,time,home_team,away_team,score
0,1928-1929,1,1,2/10/29,None,Arenas Club,Athletic Madrid,2:3
1,1928-1929,1,1,2/10/29,None,Espanyol,Real Unión,3:2
2,1928-1929,1,1,2/10/29,None,Real Madrid,Catalunya,5:0
3,1928-1929,1,1,2/10/29,None,Donostia,Athletic,1:1
4,1928-1929,1,1,2/12/29,None,Racing,Barcelona,0:2
...,...,...,...,...,...,...,...,...
47963,2021-2022,1,3,8/28/21,10:00 PM,Real Betis,Real Madrid,0:1
47964,2021-2022,1,3,8/29/21,5:00 PM,Barcelona,Getafe,2:1
47965,2021-2022,1,3,8/29/21,7:30 PM,Cádiz CF,CA Osasuna,2:3
47966,2021-2022,1,3,8/29/21,7:30 PM,Rayo Vallecano,Granada CF,4:0


In [46]:
df_first[["local_score", "visit_score"]] = df_first["score"].str.split(":", expand=True).astype(int)

In [48]:
df_first

,season,division,matchday,date,time,home_team,away_team,score,local_score,visit_score
0,1928-1929,1,1,2/10/29,None,Arenas Club,Athletic Madrid,2:3,2,3
1,1928-1929,1,1,2/10/29,None,Espanyol,Real Unión,3:2,3,2
2,1928-1929,1,1,2/10/29,None,Real Madrid,Catalunya,5:0,5,0
3,1928-1929,1,1,2/10/29,None,Donostia,Athletic,1:1,1,1
4,1928-1929,1,1,2/12/29,None,Racing,Barcelona,0:2,0,2
...,...,...,...,...,...,...,...,...,...,...
47963,2021-2022,1,3,8/28/21,10:00 PM,Real Betis,Real Madrid,0:1,0,1
47964,2021-2022,1,3,8/29/21,5:00 PM,Barcelona,Getafe,2:1,2,1
47965,2021-2022,1,3,8/29/21,7:30 PM,Cádiz CF,CA Osasuna,2:3,2,3
47966,2021-2022,1,3,8/29/21,7:30 PM,Rayo Vallecano,Granada CF,4:0,4,0


In [50]:
goles_locales = df_first.groupby("home_team")["local_score"].sum()

In [51]:
goles_locales

home_team
AD Almería         50
Alavés            362
Albacete          194
Arenas Club       145
Ath. Aviación      71
                 ... 
UD Salamanca      284
UE Lleida          12
Valencia         2973
Villarreal        651
Xerez CD           20
Name: local_score, Length: 68, dtype: int64

In [55]:
goles_visitantes = df_first.groupby("away_team")["visit_score"].sum()

In [56]:
goles_visitantes


away_team
AD Almería         22
Alavés            246
Albacete          126
Arenas Club        82
Ath. Aviación      42
                 ... 
UD Salamanca      141
UE Lleida          17
Valencia         1642
Villarreal        472
Xerez CD           18
Name: visit_score, Length: 68, dtype: int64

In [57]:
total_goles = goles_locales.add(goles_visitantes, fill_value=0).astype(int)

In [58]:
total_goles

home_team
AD Almería         72
Alavés            608
Albacete          320
Arenas Club       227
Ath. Aviación     113
                 ... 
UD Salamanca      425
UE Lleida          29
Valencia         4615
Villarreal       1123
Xerez CD           38
Length: 68, dtype: int64

In [59]:
df_total = total_goles.reset_index()
df_total.columns = ["team", "total_goals"]

# Ordenar de mayor a menor
df_total = df_total.sort_values(by="total_goals", ascending=False)

print(df_total)

               team  total_goals
10        Barcelona         6267
48      Real Madrid         6249
5          Athletic         4802
65         Valencia         4615
8   Atlético Madrid         4184
..              ...          ...
39           Lérida           41
67         Xerez CD           38
15        CD Condal           37
26      CyD Leonesa           34
64        UE Lleida           29

[68 rows x 2 columns]
